In [ ]:
import numpy as np
from collections import defaultdict

"""
Лабораторная работа 2:
1. Посмотроить алгоритм произведения матриц через MapReduce
2. Построить алгоритм линейной регрессии через MapReduce
"""

# Mapper для матрицы A
def mapper_A(matrix_A, p):
    m, n = matrix_A.shape # C[i,k] = ∑ A[i,j] * B[j,k]
    for i in range(m):
        for j in range(n):
            for k in range(p):
                yield (i, k), ('A', j, matrix_A[i, j])


# Mapper для матрицы B
def mapper_B(matrix_B, m):
    n, p = matrix_B.shape
    for j in range(n):
        for k in range(p):
            for i in range(m):
                yield (i, k), ('B', j, matrix_B[j, k])


# Shuffle — группируем по ключу
def shuffler(mapper_output, intermediate):
    for key, value in mapper_output:
        intermediate[key].append(value)


# Reducer — вычисление одного C[i,k]
def reducer(key, intermediate):
    values = intermediate[key]
    A_vals = {}
    B_vals = {}

    for matrix_type, j, val in values:
        if matrix_type == 'A':
            A_vals[j] = val
        else:
            B_vals[j] = val

    result = 0
    for j in A_vals:
        if j in B_vals:
            result += A_vals[j] * B_vals[j]

    return key, result


# Основная функция MapReduce умножения матриц
def multiply_mapreduce(matrix_A, matrix_B):
    m, n1 = matrix_A.shape
    n2, p = matrix_B.shape

    if n1 != n2:
        raise ValueError("Несовместимые размеры матриц")

    intermediate = defaultdict(list)

    # MAP A
    mapper_a_output = mapper_A(matrix_A, p)
    for item in mapper_a_output:
        shuffler([item], intermediate)

    # MAP B
    mapper_b_output = mapper_B(matrix_B, m)
    for item in mapper_b_output:
        shuffler([item], intermediate)

    # REDUCE
    result_matrix = np.zeros((m, p))
    for key in intermediate:
        i, k = key
        _, value = reducer(key, intermediate)
        result_matrix[i, k] = value

    return result_matrix

def test_matrix_multiplication():
    print("=== Тестирование произведения матриц ===")

    A = np.array([[1, 2], [3, 4], [5, 6]])
    B = np.array([[7, 8], [9, 10]])

    res_mr = multiply_mapreduce(A, B)
    res_np = A @ B

    print("MapReduce:\n", res_mr)
    print("NumPy:\n", res_np)
    print("Совпадает:", np.allclose(res_mr, res_np))


test_matrix_multiplication()


=== Тестирование произведения матриц ===
MapReduce:
 [[ 25.  28.]
 [ 57.  64.]
 [ 89. 100.]]
NumPy:
 [[ 25  28]
 [ 57  64]
 [ 89 100]]
Совпадает: True


In [6]:
import numpy as np
from collections import defaultdict
from sklearn.linear_model import LinearRegression


def mapper_xtx_xty(X, y):
    Xb = np.column_stack([np.ones(X.shape[0]), X])
    XTX_local = np.dot(Xb.T, Xb)
    XTy_local = np.dot(Xb.T, y)
    return [('summary', (XTX_local, XTy_local))]


def shuffle(mapped_data):
    grouped = defaultdict(list)
    for key, value in mapped_data:
        grouped[key].append(value)
    return grouped


def reducer(key, values):
    XTX_total = None
    XTy_total = None
    for XTX, XTy in values:
        if XTX_total is None:
            XTX_total = XTX
            XTy_total = XTy
        else:
            XTX_total += XTX
            XTy_total += XTy
    return XTX_total, XTy_total


def fit_normal_equation(X, y):
    # отправили данные в mapper
    mapped = mapper_xtx_xty(X, y)

    # сгруппировали
    grouped = shuffle(mapped)

    # reducer получил итоговые XᵀX и Xᵀy
    XTX, XTy = reducer('summary', grouped['summary'])

    # решение системы XᵀX * θ = Xᵀy
    try:
        theta = np.linalg.inv(XTX).dot(XTy)
    except:
        # если матрица вырожденная — берём псевдообратную
        theta = np.linalg.pinv(XTX).dot(XTy)

    return theta


def predict(X, theta):
    Xb = np.column_stack([np.ones(X.shape[0]), X])
    return np.dot(Xb, theta)


def test_linear_regression():

    np.random.seed(42)
    X = np.random.randn(1000, 2)
    true_theta = np.array([2, 3, -1])
    y = true_theta[0] + np.dot(X, true_theta[1:]) + 0.1 * np.random.randn(1000)

    print("Истинные коэффициенты:", true_theta)

    # наше решение
    theta = fit_normal_equation(X, y)
    print("\n1. Нормальное уравнение:")
    print("Найденные коэффициенты:", theta)
    print("Средняя ошибка:",
          np.mean(np.abs(theta - true_theta)))

    # sklearn
    print("\n3. Сравнение с sklearn:")
    lr = LinearRegression()
    lr.fit(X, y)
    theta_sklearn = np.concatenate([[lr.intercept_], lr.coef_])
    print("Sklearn коэффициенты:", theta_sklearn)
    print("Средняя ошибка:",
          np.mean(np.abs(theta_sklearn - true_theta)))


test_linear_regression()


Истинные коэффициенты: [ 2  3 -1]

1. Нормальное уравнение:
Найденные коэффициенты: [ 2.00039354  3.0027767  -0.99828491]
Средняя ошибка: 0.0016284438112461608

3. Сравнение с sklearn:
Sklearn коэффициенты: [ 2.00039354  3.0027767  -0.99828491]
Средняя ошибка: 0.0016284438112466788
